## **Introduction**

### **Requirements and Imports**

#### **Imports**

**Basic Libraries**

* `numpy` contains a large collection of mathematical functions
* `pandas` contains functions that are designed for data manipulation and data analysis



In [ ]:
import numpy as np
import pandas as pd

**Visualization Libraries**

* `matplotlib.pyplot` contains functions to create interactive plots
* `seaborn` is a library based on matplotlib that allows for data visualization
* `wordcloud` contains functions for generating wordclouds from text data

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
#from wordcloud import WordCloud
#from wordcloud import ImageColorGenerator

**Natural Language Processing Libraries**
* `re` is a module that allows the use of regular expressions
* `nltk` provides functions for processing text data
* `Counter` is from Python's collections module, which is helpful for tokenization
* `string` contains functions for string operations

In [ ]:
import re
#import nltk
#from nltk.stem import WordNetLemmatizer
#from nltk.corpus import stopwords
#from nltk.tokenize import RegexpTokenizer
#from nltk.probability import FreqDist
#from nltk.stem import WordNetLemmatizer
#from collections import Counter
import string


**Google Drive**
* `google.colab` a library that allows the colab notebook to mount the google drive

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

#### **Datasets and Files**

The following ``csv` file was used for this project:

- `RPDR Database_winner.csv` contains all the winners of the Drag Race franchise as well as the placements of each winner in certain maxi (major) challenges. This dataset also contains the final four of some Drag Race franchises as of March 31, 2023.

## **Modeling and Evaluation**

### **Modeling**

#### **Model Training**

#### **Hyperparameter Training**

### **Evalutation**

#### **Feature Importance**

## **Conclusion**